In [1]:
from selenium import webdriver
from selenium.webdriver.common.keys import Keys
from selenium.webdriver.common.action_chains import ActionChains
from bs4 import BeautifulSoup
import pandas as pd
import time

from selenium.webdriver.chrome.options import Options
from selenium.webdriver.support.ui import Select
from selenium.webdriver.common.by import By

from selenium.webdriver.support.ui import WebDriverWait
from selenium.webdriver.support import expected_conditions as EC

In [61]:
PROXY = "103.65.236.89:3128"
chrome_options = Options()
chrome_options.add_argument('--proxy-server=%s' % PROXY)
chrome = webdriver.Chrome("chromedriver.exe",chrome_options=chrome_options)
chrome.get("https://whatismyipaddress.com/")

C:\Users\581825\AppData\Local\Temp\ipykernel_17260\4187189892.py:4: DeprecationWarning: executable_path has been deprecated, please pass in a Service object
  chrome = webdriver.Chrome("chromedriver.exe",chrome_options=chrome_options)
C:\Users\581825\AppData\Local\Temp\ipykernel_17260\4187189892.py:4: DeprecationWarning: use options instead of chrome_options
  chrome = webdriver.Chrome("chromedriver.exe",chrome_options=chrome_options)


In [54]:
waktu_tunggu = 4

# chrome_options = Options()
# chrome_options.add_argument("--headless")
# driver = webdriver.Chrome("chromedriver.exe",options=chrome_options)
driver = webdriver.Chrome("chromedriver.exe")
driver2 = webdriver.Chrome("chromedriver.exe")

res = []

# Scrape
driver.get("http://www.charterbroker.aero/direct_operator_country_update.html?ops=Passenger")
for i in range(1,4):
    driver.get(f"http://www.charterbroker.aero/direct_operatorsearch.html?pagenumber={i}")
    WebDriverWait(driver, 10).until(EC.presence_of_all_elements_located((By.XPATH, "//*[@id='contentMain']/table[2]")))
    page=BeautifulSoup(driver.page_source)
    table = page.find('table',{'bgcolor':'#CCCCCC'})
    table_kecil = table.find_all('table')
    for row in table_kecil:
        td = row.find_all('td')
        _row = [row.text.strip() for row in td if row.text.strip()]
        lnk = row.find('a',href=True)['href']
        _row.append(lnk)
        
        #Check per page detail
        driver2.get(lnk)
        tel = WebDriverWait(driver2, 10).until(EC.presence_of_all_elements_located((By.XPATH, "/html/body/div[12]/div[1]/div[2]/div[3]/div/div[12]/div/div/div[1]/ul/li[1]")))
        _row.append(tel.text.strip())
        email = WebDriverWait(driver2, 10).until(EC.presence_of_all_elements_located((By.XPATH, "/html/body/div[12]/div[1]/div[2]/div[3]/div/div[12]/div/div/div[1]/ul/li[3]/a")))
        _row.append(email.text.strip())
        web = WebDriverWait(driver2, 10).until(EC.presence_of_all_elements_located((By.XPATH, "/html/body/div[12]/div[1]/div[2]/div[3]/div/div[12]/div/div/div[2]/ul/li/a")))
        _row.append(web.text.strip())
        
        res.append(_row)
df = pd.DataFrame(res,columns=['Company','Location','Link','Tel','Email','Web'])
df

C:\Users\581825\AppData\Local\Temp\ipykernel_17260\2949769415.py:6: DeprecationWarning: executable_path has been deprecated, please pass in a Service object
  driver = webdriver.Chrome("chromedriver.exe")
C:\Users\581825\AppData\Local\Temp\ipykernel_17260\2949769415.py:7: DeprecationWarning: executable_path has been deprecated, please pass in a Service object
  driver2 = webdriver.Chrome("chromedriver.exe")


AttributeError: 'list' object has no attribute 'text'

In [55]:
res

[]

In [9]:
halaman = 12

res = []
# driver.get("https://www.planespotters.net/production-list/Boeing/747")
# driver.get("https://www.planespotters.net/production-list/Airbus/A330-A340")
# driver.get("https://www.planespotters.net/production-list/Boeing/777")
# driver.get("https://www.planespotters.net/production-list/Boeing/787")
# driver.get("https://www.planespotters.net/production-list/Boeing/767")
driver.get("https://www.planespotters.net/production-list/Airbus/A350")

for i in range(halaman):
    time.sleep(3)
    WebDriverWait(driver, 10).until(EC.presence_of_all_elements_located((By.XPATH, "//*[@id='content']/div/div[3]/div[1]/div")))
    try:
        page=BeautifulSoup(driver.page_source)
        table = page.find('div',{'class':'datatable dt-outline dt-striped'})
        table_rows = table.find_all('div',{'class':'dt-tr'})
        for tr in table_rows:         
#Cari link
            lnk = tr.select_one("div:nth-of-type(2)").select_one("a")['href']
            lnk = "https://www.planespotters.net"+lnk
            row = [lnk]
            
            td = tr.find_all('div',{'class':'dt-td'})
            row.extend([tr.text.strip() for tr in td if tr.text.strip()])
            if row:
                res.append(row)
        WebDriverWait(driver, 10).until(EC.element_to_be_clickable((By.XPATH,"//*[@id='content']/div/div[1]/div/div[3]"))).click()
        WebDriverWait(driver, 3)
    except Exception:
        driver.refresh()
#     time.sleep(3)
#     iframe = wait.until(EC.element_to_be_clickable([By.CSS_SELECTOR, '#cnsw > iframe']))
#     driver.switch_to.frame(iframe)

In [10]:
df = pd.DataFrame(res,columns=['Link Detail','MSN','Model','Registration','Airline','Delivery Date','Status','Previous REG','Remark'])
df.to_csv('A350.csv')

In [78]:
b747 = pd.read_csv('B747.csv').dropna(subset='MSN')
b777 = pd.read_csv('B777.csv').dropna(subset='MSN')
a330 = pd.read_csv('A330_A340.csv').dropna(subset='MSN')
links = b747['Link Detail'].unique()
links = np.append(links, b777['Link Detail'].unique()) 
links = np.append(links, a330['Link Detail'].unique()) 
# links.append(pd.Series(a330['Link Detail']))

# type(links)

numpy.ndarray

In [79]:
# links = ["https://www.planespotters.net/airframe/boeing-747-300-n161jt-jt-power/3wnxn3","https://www.planespotters.net/airframe/boeing-747-400-b-hoo-cathay-pacific/epnd7r"]
res = []
for link in links:
    time.sleep(4)
    try:
        driver.get(link)
        WebDriverWait(driver, 10).until(EC.presence_of_all_elements_located((By.XPATH,"//*[@id='content']/div[2]/div")))
        ff = WebDriverWait(driver, 10).until(EC.presence_of_element_located((By.XPATH,"//*[@id='content']/div[1]/div/div[1]/div[1]/div[4]/div[2]"))).text
        row = [link,ff]
        if row:
            res.append(row)
    except Exception:
        driver.refresh()

NoSuchWindowException: Message: no such window: target window already closed
from unknown error: web view not found
  (Session info: chrome=108.0.5359.99)
Stacktrace:
Backtrace:
	Ordinal0 [0x0084ACD3+2075859]
	Ordinal0 [0x007DEE61+1633889]
	Ordinal0 [0x006DB7BD+571325]
	Ordinal0 [0x006C2E1A+470554]
	Ordinal0 [0x0072AA0B+895499]
	Ordinal0 [0x0073AC96+961686]
	Ordinal0 [0x00727136+880950]
	Ordinal0 [0x006FFEFD+720637]
	Ordinal0 [0x00700F3F+724799]
	GetHandleVerifier [0x00AFEED2+2769538]
	GetHandleVerifier [0x00AF0D95+2711877]
	GetHandleVerifier [0x008DA03A+521194]
	GetHandleVerifier [0x008D8DA0+516432]
	Ordinal0 [0x007E682C+1665068]
	Ordinal0 [0x007EB128+1683752]
	Ordinal0 [0x007EB215+1683989]
	Ordinal0 [0x007F6484+1729668]
	BaseThreadInitThunk [0x7679FEF9+25]
	RtlGetAppContainerNamedObjectPath [0x770A7BBE+286]
	RtlGetAppContainerNamedObjectPath [0x770A7B8E+238]


In [61]:
pd.DataFrame(res,columns=['Link Detail','First Fliht']).to_csv('first_flight.csv')

[['https://www.planespotters.net/airframe/boeing-747-300-n161jt-jt-power/3wnxn3',
  '7 Jun 1988'],
 ['https://www.planespotters.net/airframe/boeing-747-400-b-hoo-cathay-pacific/epnd7r',
  '28 Aug 1988']]

In [4]:
# Retry yang belum
driver.get('https://www.planespotters.net/production-list/Boeing/787?p=21')
WebDriverWait(driver, 10).until(EC.presence_of_all_elements_located((By.XPATH, "//*[@id='content']/div/div[3]/div[1]/div")))
try:
    page=BeautifulSoup(driver.page_source)
    table = page.find('div',{'class':'datatable dt-outline dt-striped'})
    table_rows = table.find_all('div',{'class':'dt-tr'})
    for tr in table_rows:         
#Cari link
        lnk = tr.select_one("div:nth-of-type(2)").select_one("a")['href']
        lnk = "https://www.planespotters.net"+lnk
        row = [lnk]

        td = tr.find_all('div',{'class':'dt-td'})
        row.extend([tr.text.strip() for tr in td if tr.text.strip()])
        if row:
            res.append(row)
    WebDriverWait(driver, 10).until(EC.element_to_be_clickable((By.XPATH,"//*[@id='content']/div/div[1]/div/div[3]"))).click()
    WebDriverWait(driver, 3)
except Exception:
    driver.refresh()